In [53]:
import pandas as pd
from lib import clean_column_names
import re

In [54]:
def read_csv():
    df = pd.read_csv("../data/input/reports.csv")
    
    return df

In [55]:
def split_rows_with_multiple_officers(df):
    df = (
        df.drop("officer_name", axis=1)
        .join(
            df["officer_name"]
            .str.split("and", expand=True)
            .stack()
            .reset_index(level=1, drop=True)
            .rename("officer_name"),
            how="outer",
        )
        .reset_index(drop=True)
    )
    return df


def sanitize_officer_context(df):
    df.loc[:, "officer_context"] = df.officer_context.str.lower().str.strip()   
    return df


def filter_by_role(df):
    df.loc[:, "officer_role"] = df.officer_role.str.lower().str.strip()

    officer_roles = ['officer', 'ofc.', 'police officer', 'police ofc.', 'lieutenant', 'police lieutenant', 'lt.',  'police lt.', 
                     'detective', 'det.', 'sergeant', 'sgt.', 'police sergeant', 'police sgt.', 'captain', 'cpt.', 'chief', 'inspector', 
                    'deputy', 'marshal', 'corporal', 'cpl.', 'police corporal', 'police cpl.', 'commander', 'cmdr.', 
                    'agent', 'patrolman', 'trooper', 'sheriff', "coroner", "dr.", "doctor", "p/o", "p.o." "police officer"]

    regex_pattern = '|'.join(officer_roles)
    regex_pattern = r'\b(?:' + regex_pattern + r')\b'
    regex_pattern = re.compile(regex_pattern, re.IGNORECASE)

    df = df[df['officer_role'].fillna("").str.contains(regex_pattern)]

    df.loc[:, "officer_role"] = (df
                                .officer_role.str.replace(r"^n/a(.+)", "", regex=True)
                                .str.replace(r"(.+)?presumed(.+)", "", regex=True)
                                .str.replace(r"(.+)?(not explicitly mentioned as a law enforcement officer|not a law enforcement officer)(.+)?", "", regex=True)
                                .str.replace(r"(.+)?(suspect|civilian|unclear from the context|role not specified)(.+)?", "", regex=True)
    )

    df = df[~((df.officer_role == ""))]
    return df 


def remove_titles(df):
    df.loc[:, "officer_name"] = df.officer_name.str.lower().str.strip().str.replace(r"^(sgt\.?|sergeant|lt\.?|lieutenant|cpl\.|corporal|reporting officer|detective|det\.|p\/o|officer|police officer|chief|chief of trials|police|coroner\'s|coroner\'s investigator|unit #\d+of|driver|cap|superintendent|not specified|dr\.|doctor|sheriff)", "", regex=True)
    return df 

def blocking_keys(df):
    df.loc[:, "fc"] = df.fillna("").first_name.map(lambda x: x[:3])
    df.loc[:, "lc"] = df.fillna("").last_name.map(lambda x: x[:3])
    df = df.drop_duplicates(subset=["first_name", "last_name", "officer_context", "officer_role"])

    full_names = df.first_name.str.cat(df.last_name, sep=" ")
    df = df[~((df.first_name == "") & (df.last_name == ""))]
    return df, full_names
def split_officer_name(name):
    if pd.isna(name) or not isinstance(name, str):
        return pd.Series({'first_name': '', 'middle_name': '', 'last_name': '', 'suffix': ''})
    
    name = name.strip().rstrip('.,')
    
    suffixes = {'jr', 'sr', 'ii', 'iii', 'iv', 'v'}
    
    suffix = ''
    for s in suffixes:
        if name.lower().endswith(' ' + s) or name.lower().endswith(',' + s):
            suffix = s
            name = name[:-(len(s)+1)].strip().rstrip(',')
            break
    
    parts = re.split(r',\s*|\s+', name)
    
    if ',' in name:  
        last_name = parts[0]
        first_name = parts[1] if len(parts) > 1 else ''
        middle_name = ' '.join(parts[2:]) or ''
    elif name.isupper():  
        last_name = parts[0]
        first_name = parts[1] if len(parts) > 1 else ''
        middle_name = ' '.join(parts[2:]) or ''
    else:  
        first_name = parts[0]
        last_name = parts[-1]
        middle_name = ' '.join(parts[1:-1]) or ''
    
    return pd.Series({'first_name': first_name, 'middle_name': middle_name, 'last_name': last_name, 'suffix': suffix})

def generate_uid(row, desired_length=10):
    raw_uid = abs(hash(f"{row['first_name']}{row['last_name']}{row['officer_role']}{row['officer_context']}"))
    raw_uid_str = str(raw_uid)
    if len(raw_uid_str) > desired_length:
        uid_str = raw_uid_str[:desired_length]
    else:
        uid_str = raw_uid_str.zfill(desired_length)
    
    return int(uid_str)


df = read_csv()

In [56]:
df = df.pipe(clean_column_names).pipe(split_rows_with_multiple_officers).pipe(filter_by_role).pipe(sanitize_officer_context).pipe(remove_titles)
 
name_parts = df['officer_name'].apply(split_officer_name)

df[['first_name', 'middle_name', 'last_name', 'suffix']] = name_parts

df, full_names = df.pipe(blocking_keys)

df.to_csv("../data/output/clean_review.csv", index=False)

In [ ]:
# df

In [ ]:
# df['person_uid'] = df.apply(lambda row: generate_uid(row, 10), axis=1)


In [ ]:
# df.to_csv("../data/output/clean.csv", index=False) 